Created by Joan-Marc Fisa

- Numerai: [FisaGol](https://numer.ai/fisagol)

- Twitter: [@fisagol](https://twitter.com/fisagol)


# Evaluating Financial Machine Learning Models on Numerai

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, 
and then re-execute this cell.


In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 38.0 gigabytes of available RAM

You are using a high-RAM runtime!


In [ ]:
!pip install numerapi
!pip install catboost;

     |████████████████████████████████| 66.3MB 104kB/s 


In [ ]:
##################################################################
##################### LIBRARIES ##################################
##################################################################

In [ ]:
import os
import gc
import csv
import sys
import glob
import time
from pathlib import Path
from multiprocessing import Pool

import numerapi

import scipy
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt

import tensorflow as tf
from catboost import CatBoostRegressor
from sklearn.linear_model import LinearRegression

import random
import sklearn
from sklearn import (
    feature_extraction, feature_selection, decomposition, linear_model,
    model_selection, metrics, svm, preprocessing, utils
)
from sklearn.preprocessing import StandardScaler,MinMaxScaler, OrdinalEncoder, LabelEncoder,OneHotEncoder
from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import Sequential, model_from_json, load_model
from keras.layers import Dense, Dropout, Activation,LSTM,Bidirectional, MaxPooling2D, Flatten,GRU
from keras.optimizers import SGD,Adam
from keras.regularizers import l2
from sklearn.model_selection import StratifiedKFold, GroupKFold, GridSearchCV,cross_val_score,KFold, RepeatedStratifiedKFold,train_test_split
from sklearn.metrics import log_loss, make_scorer, mean_squared_error,classification_report,accuracy_score
from keras.layers.normalization import BatchNormalization
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
import tensorflow as tf
from keras.utils import np_utils 
from sklearn import preprocessing
from keras.wrappers.scikit_learn import KerasClassifier
from xgboost import XGBRegressor 
from sklearn.cluster import KMeans
import xgboost as xgb
import matplotlib as plt
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.svm import SVC as svc
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler,MinMaxScaler

In [ ]:
'''
##################### CLASSES AND FUNCTIONS ##################################
'''

class Download():
    
    def training(self):
        import pandas as pd
        df_training = pd.read_csv("https://numerai-public-datasets.s3-us-west-2.amazonaws.com/latest_numerai_training_data.csv.xz")
        return df_training
    
    def tournament(self):
        import pandas as pd
        df_tournament = pd.read_csv("https://numerai-public-datasets.s3-us-west-2.amazonaws.com/latest_numerai_tournament_data.csv.xz")
        return df_tournament
    
class Submission():

    def make_submission(self,df_tournament_data):
        
        print("Generating Public_ID and Model_ID...")
        # Get your API keys and model_id from https://numer.ai/submit
        public_id = "7TISUDGAWEVO2B35ECOQQXU2RWXGZN3I"
        secret_key = "QJYUWIMFEEDNZ4GHUO6VSSKPMRCBFJIMJ7BZ65ESIWRN4YHGYHSRJDNL64TAG7EH"
        model_id = "d49c26a4-aa5b-4490-9d58-300c5e05d996"
        napi = numerapi.NumerAPI(public_id=public_id, secret_key=secret_key)
        print("Generated!")
        
        print("Generating DataFrame to submission...")
        predictions_df = df_tournament_data["id"].to_frame()
        predictions_df["prediction"] = df_tournament_data[PREDICTION_NAME]
        print("Generated!")
        
        print("Uploading DataFrame in Numerai...")
        # Upload your predictions
        predictions_df.to_csv("predictions.csv", index=False)
        submission_id = napi.upload_predictions("predictions.csv", model_id=model_id)
        print("DataFrame Uploaded...")
        
        return predictions_df

class Stats():

    def statics(df_tournament_data):
        
        """Validation Metrics"""
        # Check the per-era correlations on the validation set (out of sample)
        validation_data = df_tournament_data[df_tournament_data.data_type == "validation"]
        validation_correlations = validation_data.groupby("era").apply(score)
        
        validation_sharpe = validation_correlations.mean() / validation_correlations.std()
        
        return validation_sharpe
            
    def correlation(self,predictions, targets):
        ranked_preds = predictions.rank(pct=True, method="first")
        return np.corrcoef(ranked_preds, targets)[0, 1]

    def score(self,df):
        return Stats.correlation(df[PREDICTION_NAME], df[TARGET_NAME])

    def payout(self,scores):
        return scores.clip(lower=-0.25, upper=0.25)
        
        
def graph_importance(model):
    import xgboost as xgb
    fig, ax = plt.pyplot.subplots(1,1,figsize=(10,10))
    xgb.plot_importance(model, max_num_features=30, ax=ax)

def graph_grafo(model):        
    xgb.to_graphviz(model, num_trees=1)
    
def Cluster_Eras(df):

    X_clusters_eras = df
    eras = [e for e in X_clusters_eras.era.unique()]
    X_clusters_eras['eras_cluster'] = X_clusters_eras.era.str.slice(3).astype(int)    
    return X_clusters_eras

def Cluster_Kmeans(df,X,clusters):
    
    print('Call algorithm K-means')
    kmeans = KMeans(n_clusters=clusters, random_state=rand, init = 'random')
    print('Fitting algorithm K-means')
    kmeans.fit(X)
    print('Finished and Fitted')
    X_clusters_kmeans = df
    X_clusters_kmeans['k-means'] = kmeans.labels_
    return X_clusters_kmeans

def KMeans_Clustering_XGBRegressor(df,group,features):
    
    df_features_X = group[features]
    df_features_X['target'] = group.target
    X = df_features_X[df_features_X.columns[0:-1]]
    Y = df_features_X[df_features_X.columns[-1]]
    x_train,x_test,y_train,y_test = train_test_split(X,Y, test_size=0.20,random_state=rand)

    model = XGBRegressor(max_depth=5, learning_rate=0.01, n_estimators=2000, colsample_bytree=0.1, verbosity=1, nthread=6)
    model.fit(x_train, y_train)

    return model, x_train,x_test,y_train,y_test

def my_loss_fn(y_true, y_pred):
    squared_difference = tf.square(y_true - y_pred)
    return tf.reduce_mean(squared_difference, axis=-1)

def pearson_cumsom_loss(y_true, y_pred):
  
    if isinstance(y_true, pd.Series):
        y_true = y_true.values
    if isinstance(y_pred, pd.Series):
        y_pred = y_pred.values
    n = len(y_true)
    y_bar = y_true.mean()
    yhat_bar = y_pred.mean()
    c = 1 / ((y_true - y_bar) ** 2).sum().sqrt()  # constant variable
    b = ((y_pred - yhat_bar) ** 2).sum().sqrt()  # std of pred

    a_i = y_true - y_bar
    d_i = y_pred - yhat_bar
    a = (a_i * d_i).sum()
    gradient = c * (a_i / b - a * d_i / b**3)
    hessian = - (np.matmul(a_i.reshape(-1, 1), d_i.reshape(1, -1)) + np.matmul(d_i.reshape(-1, 1), a_i.reshape(1, -1))) / b ** 3 + \
              3 * a * np.matmul(d_i.reshape(-1, 1), d_i.reshape(1, -1)) / b**5 + a/(n*b**3)
    hessian = hessian - np.ones(shape=(n, n)) * a/b**3
    hessian *= c
    return -gradient, -hessian

dw = Download()


In [ ]:
##############################################################################
########################## DOWLOAD DATA ######################################
##############################################################################

In [ ]:
seed = 3
rand = np.random.seed(seed)

In [ ]:
napi = numerapi.NumerAPI(verbosity="info")

napi.download_current_dataset(unzip=True)

current_ds = napi.get_current_round()
latest_round = os.path.join('numerai_dataset_'+str(current_ds))

./numerai_dataset_240.zip: 381MB [00:08, 46.3MB/s]                           
2020-12-05 12:03:13,730 INFO numerapi.base_api: unzipping file...


In [ ]:
TOURNAMENT_NAME = "nomi"
TARGET_NAME = f"target"
PREDICTION_NAME = f"prediction"

BENCHMARK = 0
BAND = 0.2

#-----------------------------------------------------

# Submissions are scored by spearman correlation
def score(df):
    # method="first" breaks ties based on order in array
    return np.corrcoef(
        df[TARGET_NAME],
        df[PREDICTION_NAME].rank(pct=True, method="first")
    )[0, 1]

# The payout function
def payout(scores):
    return ((scores - BENCHMARK) / BAND).clip(lower=-1, upper=1)


# Read the csv file into a pandas Dataframe
def read_csv(file_path):
    with open(file_path, 'r') as f:
        column_names = next(csv.reader(f))
        dtypes = {x: np.float16 for x in column_names if
                  x.startswith(('feature', 'target'))}
    return pd.read_csv(file_path, dtype=dtypes)

In [ ]:
##################################################################
##################### LOAD DATA ##################################
##################################################################

In [ ]:
%%time
print("# Loading data...")

training_data = read_csv(os.path.join(latest_round, "numerai_training_data.csv")).set_index("id")
tournament_data = read_csv(os.path.join(latest_round, "numerai_tournament_data.csv")).set_index("id")
validation_data = tournament_data[tournament_data.data_type == "validation"]

print("# All Loaded...")

# Loading data...
# All Loaded...
CPU times: user 55.6 s, sys: 2.65 s, total: 58.2 s
Wall time: 58.2 s


In [ ]:
feature_names = [f for f in training_data.columns if f.startswith("feature")]
print(f"Loaded {len(feature_names)} features")

Loaded 310 features


In [ ]:
TRAIN_EVAL_PREFIX = "train"
VAL_EVAL_PREFIX = "val"

In [ ]:
##############################################################################
########################  DIVIDE DATA  in X and Y  ###########################
##############################################################################

In [ ]:
# NORAML DATA

X = training_data[training_data.columns[3:-1]]
Y = training_data[training_data.columns[-1]]

# ENCODER Y 2 TYPES

le = LabelEncoder()
Y_enc = le.fit_transform(Y)
dummy_y = np_utils.to_categorical(Y_enc)
df_dummy_y = pd.DataFrame(dummy_y)

#PCA WITHOUT SCALER

pca2 = PCA(n_components=125, random_state=rand)
pca_2 = pca2.fit_transform(training_data[feature_cols])
df_zero = pd.DataFrame(pca_2, columns=[feature_cols[0:125]])
X_zero = df_zero

# SCALED DATA

df_x = training_data[feature_cols]
ss = StandardScaler()
df_x[feature_cols] = ss.fit_transform(df_x[feature_cols])
X_x = df_x

# PCA WITH SCALED DATA

df_z = df_x
pca2 = PCA(n_components=125, random_state=rand)
pca_2 = pca2.fit_transform(df_z)
df_z = pd.DataFrame(pca_2, columns=[feature_cols[0:125]])
X_z = df_z

In [ ]:
##################################################################
#####################  METRICS FIRST  ############################
##################################################################

In [ ]:
def spearmanr(target, pred):
    return np.corrcoef(
        target,
        pred.rank(pct=True, method="first")
    )[0, 1]

def get_basic_per_era_metrics(df:pd.DataFrame, 
                        isVal=None, 
                        fig_name="per_era_scores.png") -> pd.Series:
  
    prefix=None
    #Some checks for deciding between training and tournament data
    if isVal:
        df = df[df.data_type == "validation"]
        prefix=VAL_EVAL_PREFIX
        print("predicting on validation...")
    else:
        df = df
        prefix=TRAIN_EVAL_PREFIX
        print("predicting on train...")

    #-----------------------------------------------------
    scores = pd.Series(dtype=float)
    preds_ = df[PREDICTION_NAME]
    scores[f"{prefix}_mean"] = preds_.mean()
    scores[f"{prefix}_std_dev"] = preds_.std()

    #-----------------------------------------------------
    #Metric Calculations
    print("getting per era scores")
    era_scores = df.groupby("era").apply(
        lambda x: spearmanr(x[TARGET_NAME], x[PREDICTION_NAME]))
    
    era_scores.sort_index(inplace=True)
    era_scores.plot(kind="bar")
    print("performance over time")
    plt.pyplot.savefig(f"{prefix}_{fig_name}")
    plt.pyplot.show()

    #-----------------------------------------------------
    scores[f"{prefix}_mean correlation"] = np.mean(era_scores)
    scores[f"{prefix}_Std. Dev."] = np.std(era_scores)
    scores[f"{prefix}_sharpe"] = np.mean(era_scores)/np.std(era_scores)

    print(scores)
    del era_scores
    del preds_
    
    gc.collect()
    return scores


In [ ]:
def get_all_metrics(model, 
                    feature_names:list=feature_names, 
                    fig_name="per_era_scores")->pd.Series:

    training_preds = model.predict(training_data[feature_names].values)
    training_data[PREDICTION_NAME] = np.array(training_preds).reshape(-1,1)

    tournament_preds = model.predict(tournament_data[feature_names].values)
    tournament_data[PREDICTION_NAME] = np.array(tournament_preds).reshape(-1,1)

    del training_preds
    del tournament_preds

    print("evaluating on training data...")
    tr_per_era_scores = get_basic_per_era_metrics(training_data, isVal=False, fig_name=fig_name)
    gc.collect()

    print("evaluating on validation data...")
    val_per_era_scores = get_basic_per_era_metrics(tournament_data, isVal=True, fig_name=fig_name)
    gc.collect()

    return pd.concat([
                      tr_per_era_scores, val_per_era_scores,
                      ])



In [ ]:
#############################################################################
#############################  CREATING SOME MODELS   #######################
#############################################################################

In [ ]:
models = dict()

#Linear model
lin_reg = LinearRegression()
models["lin_reg"] = lin_reg

#Neural Net
nn_model = tf.keras.models.Sequential([
                                       tf.keras.layers.Input(shape=(310,)),
                                       tf.keras.layers.Dense(64, activation="relu"),
                                       tf.keras.layers.Dense(1, activation="sigmoid")
])
nn_model.compile(loss="mse", optimizer="adam", metrics = [tf.keras.metrics.RootMeanSquaredError()])

models["keras_mlp_simple"] = nn_model

#Neural Net Complex

input_data = tf.keras.Input(shape=(len(feature_names),))
# tf.keras.layers.PReLU(alpha_initializer="zeros", alpha_regularizer=None, alpha_constraint=None, shared_axes=None)
# tf.keras.layers.ELU(alpha=1.0)
# tf.keras.layers.ThresholdedReLU(theta=1.0)
# tf.keras.layers.LeakyReLU(alpha=0.3)
layer = tf.keras.layers.LeakyReLU(alpha=0.1)
final_layer = tf.keras.layers.Softmax(axis=-1)
x = tf.keras.layers.Dense(len(feature_names), activation=layer)(input_data)
x = tf.keras.layers.Dense(len(feature_names) // 2, activation=layer)(x)
x = tf.keras.layers.Dense(len(feature_names) // 4, activation=layer)(x)
output = tf.keras.layers.Dense(1, activation='sigmoid')(x)
optimizer = tf.keras.optimizers.Adam( learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False, name="Adam")
nn_model_complex = tf.keras.Model(input_data, output)
# model.compile(optimizer=optimizer, loss=pearson_cumsom_loss, metrics=['mae', 'mse'])
nn_model_complex.compile(optimizer=optimizer, loss=my_loss_fn, metrics=['mae', 'mse'])


models["keras_mlp_complex"] = nn_model_complex


#CatBoost Regressor
cat_regressor = CatBoostRegressor()
models["cat_reg"] = cat_regressor

model_XGBRegressor = XGBRegressor(max_depth=10, learning_rate=0.01, n_estimators=2000, colsample_bytree=0.1, nthread=15)
models["model_XGB"] = model_XGBRegressor


del lin_reg
del nn_model
del nn_model_complex
del cat_regressor
del model_XGBRegressor

gc.collect()

14

In [ ]:
models

{'cat_reg': <catboost.core.CatBoostRegressor at 0x7f92f6b92a20>,
 'keras_mlp_complex': <tensorflow.python.keras.engine.functional.Functional at 0x7f92f67ceb70>,
 'keras_mlp_simple': <tensorflow.python.keras.engine.sequential.Sequential at 0x7f92f6b15128>,
 'lin_reg': LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False),
 'model_XGB': XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.1, gamma=0,
              importance_type='gain', learning_rate=0.01, max_delta_step=0,
              max_depth=10, min_child_weight=1, missing=None, n_estimators=2000,
              n_jobs=1, nthread=15, objective='reg:linear', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)}

In [ ]:
#############################################################################
####################   TRAINING OUR MODELS   ################################
#############################################################################

In [ ]:
for model_name in models:
    print(f"Fitting {model_name}...")

    if "keras" in model_name:
        models[model_name].fit(training_data[feature_names].values, training_data[TARGET_NAME].values, 
             batch_size=512, 
             epochs=40,
             validation_data=(validation_data[feature_names].values, validation_data[TARGET_NAME].values),
             )
    else:
        models[model_name].fit(training_data[feature_names].values, training_data[TARGET_NAME].values)

    gc.collect()

In [ ]:
models

{'cat_reg': <catboost.core.CatBoostRegressor at 0x7f92f6b92a20>,
 'keras_mlp_complex': <tensorflow.python.keras.engine.functional.Functional at 0x7f92f67ceb70>,
 'keras_mlp_simple': <tensorflow.python.keras.engine.sequential.Sequential at 0x7f92f6b15128>,
 'lin_reg': LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False),
 'model_XGB': XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.1, gamma=0,
              importance_type='gain', learning_rate=0.01, max_delta_step=0,
              max_depth=10, min_child_weight=1, missing=None, n_estimators=2000,
              n_jobs=1, nthread=15, objective='reg:linear', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)}

In [ ]:
#############################################################################
##########################    EVALUATING MODELS    ##########################
#############################################################################

In [ ]:
%%time
all_model_metrics = dict()
for model_name in models:
    
    print(f"\n----{model_name}----")
    model_metrics = get_all_metrics(models[model_name], feature_names, fig_name = f"{model_name}.png")
    all_model_metrics[model_name] = model_metrics

    gc.collect()


In [ ]:
metric_df = pd.DataFrame.from_dict(all_model_metrics)
metric_df

,lin_reg,keras_mlp_simple,keras_mlp_complex,cat_reg,model_XGB
train_mean,0.500004,0.503093,0.504430,0.499997,0.499993
train_std_dev,0.011704,0.036664,0.107161,0.024281,0.058124
train_mean correlation,0.052274,0.124772,0.396850,0.339578,0.828164
train_Std. Dev.,0.030311,0.024853,0.023751,0.023286,0.012835
train_sharpe,1.724580,5.020416,16.708468,14.582642,64.525860
val_mean,0.500001,0.503810,0.507572,0.501076,0.500710
val_std_dev,0.011691,0.039992,0.111005,0.022905,0.022037
val_mean correlation,0.016143,0.011350,0.006056,0.017243,0.022210
val_Std. Dev.,0.029660,0.019313,0.019238,0.025081,0.027271
val_sharpe,0.544282,0.587694,0.314799,0.687500,0.814405



- **`val_mean_correlation`** suggests how much your predictions are correlated to targets across `era`
- **`val_sharpe`** (mean/std) is for higher mean with lower std. dev.
- **`tournament_corr_example_preds`** shows how much your predictions are correlated with `example_predictions` (which have shown very good performance)

As you can see, Neural Net is showing better performance here based on these metrics. Now, let's compare Neural Net with default CatBoost model on more metrics


In [ ]:
##################################################################
#####################   MORE METRICS   ###########################
##################################################################

In [ ]:
TRAIN_EVAL_PREFIX = "train"
VAL_EVAL_PREFIX = "val"

#Some evaluation metrics
def ar1(x):
    return np.corrcoef(x[:-1], x[1:])[0,1]

def autocorr_penalty(x):
    n = len(x)
    p = ar1(x)
    return np.sqrt(1 + 2*np.sum([((n - i)/n)*p**i for i in range(1,n)]))

def smart_sharpe(x):
    return np.mean(x)/(np.std(x, ddof=1)*autocorr_penalty(x))

def numerai_sharpe(x):
    return ((np.mean(x) - 0.010415154) / np.std(x)) * np.sqrt(12)

def spearmanr(target, pred):
    return np.corrcoef(
        target,
        pred.rank(pct=True, method="first")
    )[0, 1]

#-----------------------------------------------------
def get_baisc_per_era_metrics(df:pd.DataFrame, 
                        isVal=None, 
                        fig_name="per_era_scores.png") -> pd.Series:
    
    prefix=None
    scores = pd.Series(dtype=float)

    preds_ = df[PREDICTION_NAME]
    #Some checks for deciding between training and tournament data
    if isVal:
        #scores["tournament_corr_example_preds"] = spearmanr(preds_, example_preds[PREDICTION_NAME])
        df = df[df.data_type == "validation"]
        prefix=VAL_EVAL_PREFIX
        print("predicting on validation...")
    else:
        df = df
        prefix=TRAIN_EVAL_PREFIX
        print("predicting on train...")

    #-----------------------------------------------------

    #Metric Calculations
    print("getting per era scores")
    era_scores = df.groupby("era").apply(
        lambda x: spearmanr(x[TARGET_NAME], x[PREDICTION_NAME]))
    
    era_scores.sort_index(inplace=True)
    era_scores.plot(kind="bar")
    print("performance over time")
    plt.pyplot.savefig(f"{prefix}_{fig_name}")
    plt.pyplot.show()

    #-----------------------------------------------------
    
    scores[f"{prefix}_mean"] = preds_.mean()
    scores[f"{prefix}_std_dev"] = preds_.std()
    scores[f"{prefix}_less_than_half"] = (preds_<0.5).mean()
    scores[f"{prefix}_less_than_mean"] = (preds_<preds_.mean()).mean()

    scores[f"{prefix}_autocorrelation"] = ar1(era_scores)
    scores[f"{prefix}_mean correlation"] = np.mean(era_scores)
    scores[f"{prefix}_Median Correlation"] = np.median(era_scores)
    scores[f"{prefix}_Variance"] = np.var(era_scores)
    scores[f"{prefix}_Std. Dev."] = np.std(era_scores)
    scores[f"{prefix}_sharpe"] = np.mean(era_scores)/np.std(era_scores)
    scores[f"{prefix}_smart sharpe"] = smart_sharpe(era_scores)
    scores[f"{prefix}_Numerai sharpe"] = numerai_sharpe(era_scores)

    print(scores)
    del era_scores
    del preds_
    gc.collect()
    return scores


In [ ]:
def neutralize(df, columns, by, proportion=1.0):
    scores = df[columns]
    exposures = df[by].values
    
    # constant column to make sure the series is completely neutral to exposures
    exposures = np.hstack((exposures, np.array([np.mean(scores)] * len(exposures)).reshape(-1, 1)))
    gc.collect()
    scores = scores - proportion * exposures.dot(np.linalg.pinv(exposures).dot(scores))
    gc.collect()
    return scores / scores.std()


def calculate_feature_exposure(df, feature_names) -> list:
    exposures = []
    for feature_name in feature_names:
        exposures.append(spearmanr(df[feature_name], df[PREDICTION_NAME]))
        
    max_feat_exposure = np.max(np.abs(exposures))
    square_sum_feature_exposure = np.sum([e**2 for e in exposures])
    feature_exposure = np.std(exposures)

    #print(max_feat_exposure, square_sum_feature_exposure)

    return [feature_exposure, max_feat_exposure, square_sum_feature_exposure]


def get_more_metrics(df, feature_names, isVal=None) -> pd.Series:
    
    more_metrics = pd.Series(dtype=float)
    metric_prefix=None
    assert PREDICTION_NAME in df.columns

    if isVal is None:
        isVal = "validation" in df["data_type"].unique() #max CPU times: user 65.1 ms

    print(isVal)
    if isVal:
        df = df[df["data_type"]=="validation"]
        metric_prefix = VAL_EVAL_PREFIX
    else:
        metric_prefix = TRAIN_EVAL_PREFIX

    assert metric_prefix is not None

    #-----------------------------------------------------

    #per-era scores
    
    print("predicting per-era scores...")
    scores_per_era = df.groupby("era").apply(
        lambda df: spearmanr(df[PREDICTION_NAME], df[TARGET_NAME]))
    
    more_metrics[f"{metric_prefix}_var"] = scores_per_era.std()

    #-----------------------------------------------------
    
    #Neutralize
    #This takes a significant amount of memory for calculation
    print(df.shape)
    print("Neutralizing...")
    df[f"neutral_{PREDICTION_NAME}"] = neutralize(df, PREDICTION_NAME, feature_names)
    feature_neutral_mean = df.groupby("era").apply(
        lambda x: spearmanr(x["neutral_"+PREDICTION_NAME].values, x[TARGET_NAME])).mean()

    more_metrics[f"{metric_prefix}_feature_neutral_mean"] = feature_neutral_mean
    gc.collect()

    #-----------------------------------------------------
    print("Calculating Feature Exposure...")
    feature_exposure, max_feat_exposure, square_sum_feature_exposure = calculate_feature_exposure(df, feature_names)

    more_metrics[f"{metric_prefix}_feat_exposure"] = feature_exposure
    more_metrics[f"{metric_prefix}_max_feat_exposure"] = max_feat_exposure
    more_metrics[f"{metric_prefix}_square_sum_feature_exposure"] = square_sum_feature_exposure


    #-----------------------------------------------------
    print("Drawdown...")
    rolling_max = (scores_per_era+1).cumprod().rolling(window=100, min_periods=1).max()
    daily_value = (scores_per_era+1).cumprod()
    max_drawdown = (rolling_max - daily_value).max()

    more_metrics[f"{metric_prefix}_max_drawdown"] = max_drawdown

    return more_metrics

In [ ]:
def get_all_metrics(model, 
                    feature_names:list=feature_names, 
                    fig_name="per_era_scores")->pd.Series:

    training_preds = model.predict(training_data[feature_names].values)
    training_data[PREDICTION_NAME] = np.array(training_preds).reshape(-1,1)

    tournament_preds = model.predict(tournament_data[feature_names].values)
    tournament_data[PREDICTION_NAME] = np.array(tournament_preds).reshape(-1,1)

    del training_preds
    del tournament_preds

    print("evaluating on training data...")
    tr_per_era_scores = get_baisc_per_era_metrics(training_data, isVal=False, fig_name=fig_name)
    tr_more_metrics = get_more_metrics(training_data, feature_names ,isVal=False)
    gc.collect()

    print("evaluating on validation data...")
    val_per_era_scores = get_baisc_per_era_metrics(tournament_data, isVal=True, fig_name=fig_name)
    val_more_metrics = get_more_metrics(tournament_data, feature_names ,isVal=True)
    gc.collect()

    return pd.concat([
                      tr_per_era_scores, val_per_era_scores,
                      tr_more_metrics, val_more_metrics,
                      ])
    

In [ ]:
#############################################################################
###########################  CREATING SOME MODELS  ##########################
#############################################################################

In [ ]:
#Linear model
lin_reg = LinearRegression()
models["lin_reg"] = lin_reg

#Neural Net
nn_model = tf.keras.models.Sequential([
                                       tf.keras.layers.Input(shape=(310,)),
                                       tf.keras.layers.Dense(64, activation="relu"),
                                       tf.keras.layers.Dense(1, activation="sigmoid")
])
nn_model.compile(loss="mse", optimizer="adam", metrics = [tf.keras.metrics.RootMeanSquaredError()])

models["keras_mlp_simple"] = nn_model

#Neural Net Complex

input_data = tf.keras.Input(shape=(len(feature_names),))
# tf.keras.layers.PReLU(alpha_initializer="zeros", alpha_regularizer=None, alpha_constraint=None, shared_axes=None)
# tf.keras.layers.ELU(alpha=1.0)
# tf.keras.layers.ThresholdedReLU(theta=1.0)
# tf.keras.layers.LeakyReLU(alpha=0.3)
layer = tf.keras.layers.LeakyReLU(alpha=0.1)
final_layer = tf.keras.layers.Softmax(axis=-1)
x = tf.keras.layers.Dense(len(feature_names), activation=layer)(input_data)
x = tf.keras.layers.Dense(len(feature_names) // 2, activation=layer)(x)
x = tf.keras.layers.Dense(len(feature_names) // 4, activation=layer)(x)
output = tf.keras.layers.Dense(1, activation='sigmoid')(x)
optimizer = tf.keras.optimizers.Adam( learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False, name="Adam")
nn_model_complex = tf.keras.Model(input_data, output)
# model.compile(optimizer=optimizer, loss=pearson_cumsom_loss, metrics=['mae', 'mse'])
nn_model_complex.compile(optimizer=optimizer, loss=my_loss_fn, metrics=['mae', 'mse'])


models["keras_mlp_complex"] = nn_model_complex


#CatBoost Regressor
cat_regressor = CatBoostRegressor()
models["cat_reg"] = cat_regressor

model_XGBRegressor = XGBRegressor(max_depth=10, learning_rate=0.01, n_estimators=2000, colsample_bytree=0.1, nthread=15)
models["model_XGB"] = model_XGBRegressor


del lin_reg
del nn_model
del nn_model_complex
del cat_regressor
del model_XGBRegressor

gc.collect()

11992

In [ ]:
models

In [ ]:
#############################################################################
####################   TRAINING MORE MODELS   ###############################
#############################################################################

In [ ]:
for model_name in models:
    print(f"Fitting {model_name}...")

    if "keras" in model_name:
        models[model_name].fit(training_data[feature_names].values, training_data[TARGET_NAME].values, 
             batch_size=512, 
             epochs=40,
             validation_data=(validation_data[feature_names].values, validation_data[TARGET_NAME].values),
             )
    else:
        models[model_name].fit(training_data[feature_names].values, training_data[TARGET_NAME].values)

    gc.collect()

In [ ]:
##############################################################################
########################    EVALUATING MODELS    #############################
##############################################################################

In [ ]:
%%time
all_model_metrics = dict()
for model_name in models:
    
    print(f"\n----{model_name}----")
    model_metrics = get_all_metrics(models[model_name], feature_names, fig_name = f"{model_name}.png")
    all_model_metrics[model_name] = model_metrics
    
    gc.collect()
    

In [ ]:
metric_df = pd.DataFrame.from_dict(all_model_metrics)
metric_df

In [ ]:
##############################################################################
######################## MAKE PREDICTIONS ####################################
##############################################################################

In [ ]:
%%time
print("Generating predictions on training data...")
training_preds = model.predict(training_data[feature_names].astype(np.float32))
training_data[PREDICTION_NAME] = training_preds
gc.collect()

print("Generating predictions on tournament data...")
tournament_preds = model.predict(tournament_data[feature_names].astype(np.float32))
tournament_data[PREDICTION_NAME] = tournament_preds

In [ ]:
# Check the per-era correlations on the training set (in sample)
train_correlations = training_data.groupby("era").apply(score)
print(f"On training the correlation has mean {train_correlations.mean()} and std {train_correlations.std()}")
print(f"On training the average per-era payout is {payout(train_correlations).mean()}")

# Check the per-era correlations on the validation set (out of sample)
validation_data = tournament_data[tournament_data.data_type == "validation"]
validation_correlations = validation_data.groupby("era").apply(score)
print(f"On validation the correlation has mean {validation_correlations.mean()} and "
        f"std {validation_correlations.std()}")
print(f"On validation the average per-era payout is {payout(validation_correlations).mean()}")

In [ ]:
# FEAT_EXPOSURE: How much your model is correlated to the features across era
feature_exposures = validation_data[feature_names].apply(
    lambda d: correlation(validation_data[PREDICTION_NAME], d), axis=0
)
max_per_era = validation_data.groupby("era").apply(
    lambda d: d[feature_names].corrwith(d[PREDICTION_NAME]).abs().max()
)
max_feature_exposure = max_per_era.mean()
print(f"Max Feature Exposure: {max_feature_exposure}")

In [ ]:
print("Generating predictions...")
    
torunament_features = tournament_data[feature_cols]
enc_Y_tournament = model.predict(torunament_features)
df_enc_Y_tournament = pd.DataFrame(enc_Y_tournament)
tournament_data[PREDICTION_NAME] = df_enc_Y_tournament

print("Predictions Generated...")

In [ ]:
##############################################################################
######################### MAKE SUBMISSION #################################### 
##############################################################################

In [ ]:
tournament_data[PREDICTION_NAME].to_csv(f"{TOURNAMENT_NAME}_{current_ds}_submission.csv")

In [ ]:
# NameOfYourAI
public_id = "7TISUDGAWEVO2B35ECOQQXU2RWXGZN3I"
secret_key = "QJYUWIMFEEDNZ4GHUO6VSSKPMRCBFJIMJ7BZ65ESIWRN4YHGYHSRJDNL64TAG7EH"
model_id = "d49c26a4-aa5b-4490-9d58-300c5e05d996"
napi = numerapi.NumerAPI(public_id=public_id, secret_key=secret_key)

In [ ]:
submission_id = napi.upload_predictions(f"{TOURNAMENT_NAME}_{current_ds}_submission.csv", model_id=model_id)

In [ ]:
print("Generating Public_ID and Model_ID...")
# Get your API keys and model_id from https://numer.ai/submit
public_id = "7TISUDGAWEVO2B35ECOQQXU2RWXGZN3I"
secret_key = "QJYUWIMFEEDNZ4GHUO6VSSKPMRCBFJIMJ7BZ65ESIWRN4YHGYHSRJDNL64TAG7EH"
model_id = "d49c26a4-aa5b-4490-9d58-300c5e05d996"
napi = numerapi.NumerAPI(public_id=public_id, secret_key=secret_key)
print("Generated!")

print("Generating DataFrame to submission...")
predictions_df = tournament_data["id"].to_frame()
predictions_df["prediction"] = tournament_data[PREDICTION_NAME]
print("Generated!")

print("Uploading DataFrame in Numerai...")
# Upload your predictions
predictions_df.to_csv("predictions.csv", index=False)
submission_id = napi.upload_predictions("predictions.csv", model_id=model_id)
print("DataFrame Uploaded...")

In [ ]:
##############################################################################
########################  SAVE AND LOAD THE MODEL  ###########################
##############################################################################

In [ ]:
# SAVE MODEL
model.save_weights('numeraiThe_Model.h5', overwrite=True, save_format=None, options=None)

In [ ]:
# LOAD MODEL
model.load_weights('numeraiThe_Model.h5', by_name=False, skip_mismatch=False, options=None)